In [1]:
%pip install -e ..
%pip install marvelous@git+https://github.com/end-to-end-mlops-databricks-3/marvelous@0.1.0

Obtaining file:///Users/tommy/Git/marvelous-databricks-course-tommydejong
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fifa-players (pyproject.toml) ... done
  Created wheel for fifa-players: filename=fifa_players-0.0.1-0.editable-py3-none-any.whl size=1729 sha256=69d01923a55a1d490b44c77abac520abccc1f9a549aeb7a08dddf36132a7c64b
  Stored in directory: /private/var/folders/9h/6psrv7qs3f55f1q8t1s_qcgh0000gn/T/pip-ephem-wheel-cache-g7_2wt0m/wheels/20/55/00/003fb2b8b27756b0d4a1b8e6cd17bbc4e513903f61df41e753
Successfully built fifa-players
  Attempting uninstall: fifa-players
    Found existing installation: fifa-players 0.0.1
    Uninstalling fifa-players-0.0.1:
      Successfully uninstalled fifa-players-0.0.1
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/en

In [2]:
%restart_python

UsageError: Line magic function `%restart_python` not found.


In [3]:
# Databricks notebook source

import mlflow
import os
import sys
sys.path.append('../src/')

from pyspark.sql import SparkSession

from fifa_players.config import ProjectConfig, Tags
from fifa_players.models.basic_model import BasicModel

from dotenv import load_dotenv
from marvelous.common import is_databricks

# COMMAND ----------
# If you have DEFAULT profile and are logged in with DEFAULT profile,
# skip these lines

if not is_databricks():
    load_dotenv()
    profile = os.environ["PROFILE"]
    mlflow.set_tracking_uri(f"databricks://{profile}")
    mlflow.set_registry_uri(f"databricks-uc://{profile}")


config = ProjectConfig.from_yaml(config_path="../project_config.yml", env="prd")
spark = SparkSession.builder.getOrCreate()
tags = Tags(**{"git_sha": "abcd12345", "branch": "week2"})

# COMMAND ----------
# Initialize model with the config path
basic_model = BasicModel(config=config, tags=tags, spark=spark)

# COMMAND ----------
basic_model.load_data()
basic_model.prepare_features()

# COMMAND ----------
# Train + log the model (runs everything including MLflow logging)
basic_model.train()
basic_model.log_model()

# COMMAND ----------
run_id = mlflow.search_runs(
    experiment_names=["/Shared/house-prices-basic"], filter_string="tags.branch='week2'"
).run_id[0]

model = mlflow.sklearn.load_model(f"runs:/{run_id}/lightgbm-pipeline-model")

# COMMAND ----------
# Retrieve dataset for the current run
basic_model.retrieve_current_run_dataset()

# COMMAND ----------
# Retrieve metadata for the current run
basic_model.retrieve_current_run_metadata()

# COMMAND ----------
# Register model
basic_model.register_model()

# COMMAND ----------
# Predict on the test set

test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").limit(10)

X_test = test_set.drop(config.target).toPandas()

predictions_df = basic_model.load_latest_model_and_predict(X_test)
# COMMAND ----------

KeyError: 'PROFILE'